In [4]:
import pandas as pd

In [8]:
def process_products(sf_file, robo_file, start_date, end_date, output_file):
    """
    Compare products from Salesforce and Robo files, filter by date range, and merge the results.
    """
    import pandas as pd

    # Load Salesforce file
    sf = pd.read_excel(sf_file)
    sf.columns = sf.columns.str.strip()

    # Load Robo file
    if robo_file.endswith(".csv"):
        robo = pd.read_csv(robo_file)
    else:
        robo = pd.read_excel(robo_file)
    robo.columns = robo.columns.str.strip()

    # Determine which date column exists in Robo
    possible_date_columns = ["ProductDate", "Product Date", "_ProductDate"]
    robo_date_col = None
    for col in possible_date_columns:
        if col in robo.columns:
            robo_date_col = col
            break

    if robo_date_col is None:
        print("Robo columns:", robo.columns.tolist())
        raise KeyError("Robo file does not have any expected date column (ProductDate / Product Date / _ProductDate)")

    print("Using Robo date column:", robo_date_col)

    # Parse dates
    sf["Product Date"] = pd.to_datetime(sf["Product Date"], format="%m/%d/%Y, %I:%M %p", errors="coerce")
    robo[robo_date_col] = pd.to_datetime(robo[robo_date_col], errors="coerce")

    # Convert date range to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter by date range
    sf = sf[(sf["Product Date"] >= start_date) & (sf["Product Date"] <= end_date)]
    robo = robo[(robo[robo_date_col] >= start_date) & (robo[robo_date_col] <= end_date)]

    # Print shapes for debug
    print("Shape of sf:", sf.shape)
    print("Shape of robo:", robo.shape)

    # Ensure ProductId exists in Robo
    if "ProductId" not in robo.columns:
        raise KeyError("Robo file does not have 'ProductId' column")

    # Clean IDs
    robo["ProductId"] = robo["ProductId"].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None)
    sf["Roboticket ID"] = sf["Roboticket ID"].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None)

    # Merge on Roboticket ID <-> ProductId
    merged = pd.merge(
        sf,
        robo,
        how="outer",
        left_on="Roboticket ID",
        right_on="ProductId",
        indicator=True,
        suffixes=("_SF", "_Robo")
    )

    # Rename date columns
    merged = merged.rename(columns={
        "ProductId": "ProductId_Robo",
        "Product Date": "Product Date SF",
        robo_date_col: "Product Date Robo"
    })

    columns_to_output = [
        "ProductId_Robo",
        "Roboticket ID",
        "Product Name",
        "ProductName",
        "Product Date SF",
        "Product Date Robo",
        "ProductType",
        "Product Type",
        "StadiumId",
        "Stadium ID",
        "_merge"
    ]

    # Only include columns that exist
    columns_to_output = [col for col in columns_to_output if col in merged.columns]
    merged = merged[columns_to_output]

    # Rename merge labels
    merged["_merge"] = merged["_merge"].replace({
        "left_only": "exists_in_sf",
        "right_only": "exists_in_robo"
    })

    # Split subsets
    only_in_sf = merged[merged["_merge"] == "exists_in_sf"]
    only_in_robo = merged[merged["_merge"] == "exists_in_robo"]
    both = merged[merged["_merge"] == "both"]

    # Write to Excel
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        both.to_excel(writer, sheet_name="Both", index=False)
        only_in_sf.to_excel(writer, sheet_name="Exists_in_SF", index=False)
        only_in_robo.to_excel(writer, sheet_name="Exists_in_Robo", index=False)

    print(f"Results saved to {output_file}")

    return {
        "sf": sf,
        "robo": robo,
        "merged": merged,
        "only_in_sf": only_in_sf,
        "only_in_robo": only_in_robo,
        "both": both
    }

In [9]:
# Example usage
sf_file = "New Products Report-2025-07-02-04-21-18.xlsx"
robo_file = "products_updated.xlsx"
start_date = "2015-01-01"
end_date = "2025-12-31"
output_file = "Processed_Products.xlsx"

results = process_products(sf_file, robo_file, start_date, end_date, output_file)

# Access the results
sf_filtered = results["sf"]
robo_filtered = results["robo"]
merged = results["merged"]
only_in_robo = results["only_in_robo"]
only_in_sf = results["only_in_sf"]
both = results["both"]

print("Shape of merged:", merged.shape)


Using Robo date column: _ProductDate
Shape of sf: (236, 5)
Shape of robo: (160, 7)
Results saved to Processed_Products.xlsx
Shape of merged: (236, 11)


/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_48920/3106720883.py:93: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  merged["_merge"] = merged["_merge"].replace({


In [10]:
print('Shape of only_in_robo:', only_in_robo.shape)
only_in_robo.to_excel('Only_in_Robo.xlsx', index=False)
only_in_robo

Shape of only_in_robo: (0, 11)


,ProductId_Robo,Roboticket ID,Product Name,ProductName,Product Date SF,Product Date Robo,ProductType,Product Type,StadiumId,Stadium ID,_merge


In [11]:
print('Shape of only_in_sf:', only_in_sf.shape)
only_in_sf

Shape of only_in_sf: (76, 11)


,ProductId_Robo,Roboticket ID,Product Name,ProductName,Product Date SF,Product Date Robo,ProductType,Product Type,StadiumId,Stadium ID,_merge
0,NaN,10,Virtual Card Parking,NaN,2024-10-05 17:00:00,NaT,NaN,Voucher,NaN,1.0,exists_in_sf
1,NaN,1063,פרסום: דף העסקים של הפועל 2025/26,NaN,2025-06-09 17:00:00,NaT,NaN,Merchndise,NaN,NaN,exists_in_sf
2,NaN,1331,שלב הבתים משחק 1: עירוני נס ציונה,NaN,2024-09-10 17:00:00,NaT,NaN,Match,NaN,1.0,exists_in_sf
4,NaN,1595,שלב הבתים משחק 3: הפועל גליל עליון,NaN,2024-09-17 17:00:00,NaT,NaN,Match,NaN,1.0,exists_in_sf
6,NaN,1628,Regular Season: Round 1: Valencia Basket 🚗,NaN,2024-09-24 17:00:00,NaT,NaN,Match,NaN,1.0,exists_in_sf
...,...,...,...,...,...,...,...,...,...,...,...
220,NaN,532,STtest,NaN,2024-10-05 17:00:00,NaT,NaN,Voucher,NaN,1.0,exists_in_sf
227,NaN,566,testv3,NaN,2024-10-05 17:00:00,NaT,NaN,Voucher,NaN,1.0,exists_in_sf
232,NaN,62,מנוי לעונת 2024/2025,NaN,2024-06-16 17:00:00,NaT,NaN,SeasonTicket,NaN,1.0,exists_in_sf
233,NaN,799,הסעות למשחקי הבית - מחוץ לחומות,NaN,2024-12-18 16:00:00,NaT,NaN,Merchndise,NaN,NaN,exists_in_sf
